# Functions

## Topics
- How to declare a function
- Duck-typing in Julia
- Mutating vs. non-mutating functions
- Some higher-order functions

## Functions in a functional language
Functions are the building blocks of Julia code, acting as the subroutines, procedures, blocks, and similar structural concepts found in other programming languages.

- A function's job is to take a tuple of values as an argument list and return a value. 
- If the arguments contain mutable values like arrays, the array can be modified inside the function. 
    - By convention, an exclamation mark (!) at the end of a function's name indicates that the function may modify its arguments.

## How to declare a function

Julia gives us a few different ways to write a function. The first (and most standard) requires `function` and `end` keywords.

In [ ]:
function sayhi(name)
    println("Hi $name, nice to meet you!")
end

sayhi("R2D2")

In [ ]:
function f(x)
    return x^2
end

f(42)

## Single line function definitions
Alternatively, we could have spared a few lines of code and written:

In [ ]:
sayhi2(name) = println("Hi $name, nice to meet you!")

In [ ]:
f2(x) = x^2

## Anonymous functions
Or we could have declared them as "anonymous" functions as

In [ ]:
sayhi3 = name -> println("Hi $name, nice to meet you!")

In [ ]:
f3 = x -> x^2

## Duck-typing in Julia
*"If it quacks like a duck, it's a duck."*

Julia functions will just work on whatever input makes sense. For example, `sayhi` works also with the name written as an integer:

In [ ]:
sayhi(55595472)

And `f` will work on a matrix

In [ ]:
A = rand(3,3)

In [ ]:
f(A)

## Mutating vs. non-mutating functions
By convention, functions followed by `!` alter their contents and functions lacking `!` do not.

For example, let's look at the difference between `sort` and `sort!`.

In [ ]:
v = [3,5,2]

In [ ]:
sort(v)

In [ ]:
v

`sort(v)` returns a sorted array that contains the same elements as `v`, but `v` is left unchanged.

On the other hand, when we run `sort!(v)` the content of `v` is really modified.

In [ ]:
sort!(v)

In [ ]:
v

## Epidemic Simulation


Let's start with an interaction function. In this one, infected cells will always infect any other cell.

In [ ]:
# Copy necessary things from the previous session

"Enumerate possible states of a single cell"
@enum InfectionStatus uninfected infected dead recovered

"Data structure containing the infection status of a cell"
mutable struct Cell
    status::InfectionStatus
    infection_time::Int8
end

# Create an infected and an uninfected cell
cell1 = Cell(uninfected, 0)
cell2 = Cell(infected, 0)

In [ ]:
"""Simulate an interaction between two cells. In the other cell is
   infected, it may infect the this cell.
"""
function interact!(this_cell::Cell, other_cell::Cell)
    if this_cell.status == uninfected && other_cell.status == infected
        this_cell.status = infected
        this_cell.infection_time = 0
    end
end

In [ ]:
println(cell1)
interact!(cell1, cell2)
println(cell1)

## Some higher-order functions: `map`

`map` is a "higher-order" function in Julia that takes a *function* as one of its input arguments. `map` then applies that function to every element of the data structure you pass. 

For example
```julia
map(f, [1,2,3])
```
will correspond to
```julia
[f(1), f(2), f(3)]
```

In [ ]:
map(f, [1,2,3])

## Some higher-order functions: `broadcast`
`broadcast` is another higher-order function like `map`. `broadcast` is actually a generalization, so it can do what `map` but also more!

Syntax is the same
```julia
broadcast(f, [1,2,3])
```

And so we have again applied f (squared) to all elements of `[1,2,3]`. 

In [ ]:
broadcast(f, [1,2,3])

## Broadcasting (or vectorizing)
Some syntactic sugar for calling `broadcast` is to place `.` between the name of the function you want to broadcast and its input arguments. 
        
For example
```julia
broadcast(f, [1,2,3])
```
is the same as
```julia
f.([1,2,3])
```

Note that this is not the same as `f([1,2,3])` because we can not square a vector!

Let's try broadcasting for a matrix `A`

In [ ]:
A = [1 2 3;
     4 5 6;
     7 8 9]

In [ ]:
f(A)

In [ ]:
B = f.(A)

## Dot syntax for vectorization
The dot syntax allows to write complex compound **elementwise** expressions in a way that looks natural/closer to mathematical notation. 

For example:

In [ ]:
A + 2 .* f.(A) ./ A

Instead of the more nasty looking version with `broadcast` as

In [ ]:
broadcast(x-> x + 2 * f(x) / x, A) 

# Extra:

### Advanced: SIMD vectorization
Vectorization is discussed more in the bonus notebook about SIMD vectorization.

In short, the topic is quite technical but you should rest assured that the dot syntax actually works quite well to make your code easy to read **and** fast to run.

## Macros
Finally, let's touch the metaprogramming capabilities of Julia. 

Since metaprogramming is a whole another topic (see bonus notebook) we will only cover the very basics of something you might encounter when dealing with Julia code: macros.

Macros provide a method to include generated code in the final body of a program. A macro maps a tuple of arguments to a returned **expression**, and the resulting expression is compiled directly.

This means that macros can change how functions work, hence the *meta* in metaprogramming.

In [ ]:
macro sayhello()
    return :( println("Hello, world!") )
end

## Macro invocation
Macros are invoked with the following general syntax:
```julia
@name expr1 expr2 ...
@name(expr1, expr2, ...)
```

In [ ]:
@sayhello

## Hold up: why macros?
So, why do macros exist?

Macros are necessary because they execute when code is parsed, therefore, macros allow the programmer to generate and include fragments of customized code before the full program is run. To illustrate the usefullness, consider the following example:

In [ ]:
@time A = rand(10^3, 10^3)

In [ ]:
a = [1,2]
b = [1 2 3; 4 5 6]
a .+ b